In [4]:
%cd /scratch/bruingjde/SNAM2021-code/

import os
import typing

import joblib
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import scipy
import seaborn as sns
import sklearn.linear_model
import sklearn.model_selection
import sklearn.metrics
import sklearn.pipeline
import sklearn.preprocessing
from tqdm.auto import tqdm

import tlp

# Not used because of time considerations

/scratch/bruingjde/SNAM2021-code


In [19]:
# Delete!!
for entry in os.scandir('data'):
  filename = os.path.join(entry, 'features', 'CCN.pkl')
  if os.path.isfile(filename): os.remove(filename)

In [20]:
# Check
entries = sorted(os.scandir('data'), key=lambda x: x.name)
for entry in entries:
  if not os.path.isfile(os.path.join(entry, 'features', 'CCN.pkl')): print(entry)

<DirEntry '01'>
<DirEntry '02'>
<DirEntry '03'>
<DirEntry '04'>
<DirEntry '05'>
<DirEntry '06'>
<DirEntry '07'>
<DirEntry '08'>
<DirEntry '09'>
<DirEntry '10'>
<DirEntry '11'>
<DirEntry '12'>
<DirEntry '13'>
<DirEntry '14'>
<DirEntry '15'>
<DirEntry '16'>
<DirEntry '17'>
<DirEntry '18'>
<DirEntry '19'>
<DirEntry '20'>
<DirEntry '21'>
<DirEntry '22'>
<DirEntry '23'>
<DirEntry '24'>
<DirEntry '25'>
<DirEntry '26'>
<DirEntry '27'>
<DirEntry '28'>
<DirEntry '29'>
<DirEntry '30'>


In [15]:
def ccn(path: str, verbose: bool = False, feature_name='CCN', position=None):
  # Check if file exists
  feature_path = os.path.join(path, 'features')
  file = os.path.join(feature_path, f'{feature_name}.pkl')
  if os.path.isfile(file): return 
  
  os.makedirs(feature_path, exist_ok=True)

  # Read in
  edgelist_mature_file = os.path.join(path, 'edgelist_mature.pkl')
  if verbose: print(f'Read {edgelist_mature_file}')
  edgelist_mature = pd.read_pickle(edgelist_mature_file)
  
  instances_file = os.path.join(path, 'instances_sampled.npy')
  if verbose: print(f'Read {instances_file}')  
  instances_sampled = np.load(instances_file)

  # Get all unique nodes in instances
  nodes = np.unique(instances_sampled[:,0])

  # Get edgelist
  G = nx.from_pandas_edgelist(edgelist_mature)
  
  # Calculate
  # Source: https://networkx.org/documentation/stable/_modules/networkx/algorithms/link_prediction.html#common_neighbor_centrality
  shortest_path = {node: nx.single_source_shortest_path(G, node) for node in tqdm(nodes, disable=not verbose, position=position)}
  alpha = .8
  def predict(u, v):
      return alpha * len(list(nx.common_neighbors(G, u, v))) + (1 - alpha) * (
          G.number_of_nodes() / (len(shortest_path[u][v]) - 1)
      )
  
  scores = [predict(u, v) for u, v in tqdm(instances_sampled, disable=not verbose, position=position)]
  
  # Store
  result = {tlp.Experiment(feature=feature_name, time_aware=False): np.array(scores)}
  joblib.dump(result, os.path.join(feature_path, f'{feature_name}.pkl'))

In [18]:
entries = sorted(os.scandir('data'), key=lambda x: x.name)
for entry in tqdm(entries):
  ccn(entry, verbose=True)

  0%|          | 0/30 [00:00<?, ?it/s]

Read data/01/edgelist_mature.pkl
Read data/01/instances_sampled.npy


  0%|          | 0/15464 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [16]:
entries = sorted(os.scandir('data'), key=lambda x: x.name)
tlp.ProgressParallel(n_jobs=len(entries), total=len(entries))(
  joblib.delayed(ccn)(entry.path, verbose=True, position=position) for position, entry in enumerate(entries)
)

  0%|          | 0/30 [00:00<?, ?it/s]

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

The exit codes of the workers are {SIGKILL(-9)}